# Weather Station Data - cleaned final version 
## Level 1 QC

In [1]:
#import external libraries
import pandas as pd
import os
#pd.options.mode.chained_assignment =  # default='warn'; chained index warning
import numpy as np
import imp
import datetime
import pytz

#plotting
import matplotlib.pyplot as plt
%matplotlib notebook

#import self-written libraries
import LVL1
from settings import Glacier, Station, base_path, temp_columns, precip_columns, wind_col, out_columns
import CleanWxData as wx

In [2]:
#Do you want to display plots (slow for large datasets)
show_plots=True #True= show plots
frac=0.1 #fraction of data to show in plots (0.3 = 30%)

** Define Location-Specific Baseline Settings **

In [3]:
#path to level 0 data
file_label='_15min'
yr='all' #either "all" or the year you want
pth=base_path + 'Data/'+Glacier+ r"/AllYears/Wx/LVL0/" + Glacier.lower()+ Station+ file_label+"_"+yr+".csv"
    
#time format and column
local_time_column_name='local_time'
date_format='%Y/%m/%d %H:%M'
timezone='America/Anchorage' #choose from pytz.all_timezones

#directory to save output data
save_pth=base_path +'Data/' +Glacier+ r"/AllYears/Wx/LVL1/" + Glacier.lower()+Station+file_label+ yr+".csv"


#Need to go back and make separate settings files or something, to house different varieties...
if Glacier + Station=='Wolverine370':
    precip_columns=['TipCumulative']

### Data import
- read in data 
- set timezone
- set time as index

In [4]:
#read in level0 CSV
if Glacier +Station == "Gulkana1725":
    pth=base_path + "Data/Gulkana/AllYears/Wx/LVL0/gulkana1725_15min_all_gaps_filled.csv"

if (Glacier + Station == 'Wolverine990') | (Glacier + Station =='Gulkana1480'):
    pth=base_path +r"Data/" + Glacier + r'/AllYears/Wx/LVL0/' + Glacier.lower() + Station+ "_15min_NWIS_1990s_added.csv"
      
dat=pd.read_csv(pth)
print("read data from: "+ pth)

#Set local time to dataframe index
dat['local_time']=pd.to_datetime(dat[local_time_column_name], format=date_format)
dat=dat.set_index('local_time')
#dat=dat.tz_localize(timezone, ambiguous='infer')

read data from: Q:/Project Data/GlacierData/Benchmark_Program/Data/JuneauIcefield/AllYears/Wx/LVL0/juneauicefield2121_15min_all.csv


In [5]:
#Determine if dataset contains precip data (may want to revisit this later)
precip_present=len(precip_columns)>0
temp_present=len(temp_columns)>0
wind_present=len(wind_col)>0

In [6]:
dat=dat.sort_index() #ensure data is sorted by time

Station-specific Data Adjustments

In [7]:
# Wolverine990 and Gulkana 1480: Prior to April 2014, asp. temperature only aspirated on the hour; other msmts set to NAN
if Glacier+Station=="Wolverine990":
    dat.loc[(dat.index<'2014/04/24 00:00:00') & (dat.index.minute!=0), 'TAspirated1']=np.nan
    dat.loc[(dat.index<'2014/04/24 00:00:00') & (dat.index.minute!=0), 'TAspirated2']=np.nan
    
if Glacier+Station=="Gulkana1480":
    dat.loc[(dat.index<'2014/04/18 00:00:00') & (dat.index.minute!=0), 'TAspirated1']=np.nan
    dat.loc[(dat.index<'2014/04/18 00:00:00') & (dat.index.minute!=0), 'TAspirated2']=np.nan

** Remove malfunctioning sensors manually **
* done separately for each timestep entered on look-up table

In [8]:
#read in table listing sensors and timeperiods that need to be modified
pth_bad_sensor_dates=base_path+ r"Metadata\weather"
try:
    bad_sensor_dates_dat=pd.read_excel(os.path.join(pth_bad_sensor_dates, Glacier+"_SensorLog.xlsx"),sheetname=Glacier+Station)
    bad_sensor_dates_dat
except:
    print("no manual QC log exists for this site")
    bad_sensor_dates_dat=pd.DataFrame()
    

no manual QC log exists for this site


In [9]:
#Set bad data to NAN. Move sensor data to correct column, if neccessary.
dat=LVL1.remove_malfunctioning_sensor_data(dat, bad_sensor_dates_dat)
print("Removed")

Removed


In [10]:
# #Time subset for trouble-shooting
# dat=dat['2004':'2006']
# for precip_col in precip_columns:
#     if pd.notnull((dat[precip_col][0])):
#         dat[precip_col]=dat[precip_col]-dat[precip_col][0]
#     else:
#         print("first val in "+ precip_col + " is null")

** Set hard-coded cutoff values** (station specific)

In [11]:
#Generic Settings
low_temp_cutoff=-40.0 #deg. C
high_temp_cutoff=30.0

obvious_error_precip_cutoff=60. #if 60 mm (6 cm) in 2 hrs, obviously an error. (7/2016 has a drain of only 24 cm)

#Glacier-specific settings; can also adjust generic settings given above here
if (Glacier+Station=="Wolverine990"):    
    Stage_dies='2015/09/01 06:30:00' #time at which instrument was removed
    TPG_born='2012/10/02 00:00:00' #time at which instrument was removed
        
if (Glacier + Station== "Gulkana1480"):    
    Stage_dies='2015/09/25 11:30:00' #time at which instrument was removed
    TPG_born='2011/07/15 18:30:00' #time at which instrument was removed
    
if (Glacier + Station== "Gulkana1725"):    
    TPG_born='2014-08-26 16:00:00'
    
if (Glacier + Station== "Wolverine1420") | (Glacier+Station=="Gulkana1920") : #these are the 2 ridgetop JWS sites
    print('normal settings good')
    
#Save unaltered data for later comparrison with edits
dat_original=dat.copy() 

In [12]:
#Convert units in precipitation columns from meters to mm
if precip_present:
    for precip_col in precip_columns:
        dat.loc[:,precip_col]=dat[precip_col]*1000

In [13]:
if precip_present:
    #Set gages to NAN, when not installed (single gauge style does not cover period of record)
    for precip_col in precip_columns:
        if dat[precip_col].isnull().all():
            print("no data in "+ precip_col)
            continue
        if 'TPG' in precip_col.upper(): #this is the current precip gage
            dat.loc[:TPG_born, precip_col]=np.nan
        if 'stage' in precip_col.lower(): #this is old precip gage
            dat.loc[Stage_dies:, precip_col]=np.nan

** Adjust for size of stage gage orifice (x5) **
- this is a legacy issue; gage orifice used is bigger than instrument design. Thus, precip recorded on logger is 1/5 of true

In [14]:
if precip_present:
    if 'StageCumulative' in dat.columns:
        dat['StageCumulative']=dat.StageCumulative*5 #legacy issue; see comment above

** Clean Temperature Data**

In [15]:
if temp_present:
    #Temperature
    for temp_col in temp_columns:
        temps=dat.loc[:,temp_col]
        dat.loc[:,temp_col]=LVL1.remove_error_temperature_values(temps, low_temp_cutoff, high_temp_cutoff)

#### Plot temperature data, before and after

In [16]:
if show_plots:
    plt.figure()
    LVL1.plot_comparrison(df_old=dat_original, df_new=dat, data_col_name='Tpassive1')

<IPython.core.display.Javascript object>

** Clean Wind Speed Data **

In [17]:
#Remove wind speed data where problems with riming (or other falls to 0) are present
if wind_present:
    dat=wx.clean_wind_speed_data(dat, wind_col)

In [18]:
if show_plots and wind_present:
    plt.figure()
    LVL1.plot_comparrison(df_old=dat_original, df_new=dat, data_col_name=wind_col)

#### Clean Wind Direction Data

In [19]:
if wind_present:
    if wind_dir_columns[0] in dat.columns:
        for col in wind_dir_columns:
            dat.loc[dat[col]>360, col]=np.nan #Nonsensical; at wolverine990, this is only 3 days; results from mis-pogramming of logger

In [20]:
#Correct direction during periods when sensor not aligned to true north
if Glacier + Station == 'Wolverine990': 
    og_dir=dat.loc['2016-09-09 11': '2017-07-23', 'VecAvgWindDir'].copy()
    adj_dir=og_dir-215 # this is 360-145, as indicated by Louis's notes
    adj_dir[adj_dir<0]=adj_dir[adj_dir<0]+360 #convert negative angles to unit circle
    dat.loc['2016-09-09 11': '2017-07-23', 'VecAvgWindDir']=adj_dir

    og_dir=dat.loc['2017-07-23': '2017-09-10', 'VecAvgWindDir'].copy()
    adj_dir=og_dir-335
    adj_dir[adj_dir<0]=adj_dir[adj_dir<0]+360 #convert negative angles to unit circle
    dat.loc['2017-07-23': '2017-09-10', 'VecAvgWindDir']=adj_dir

** Plot original precipitation data**

In [21]:
#Precip
if precip_present:
    dat[precip_columns].plot()

In [22]:
# dat_original=dat_original['2002':'2003-06']
# dat=dat['2002':'2003-06']

** Remove Gauge Drain and Fill Maintenance **

In [23]:
if precip_present:
    #Early stage gage measurements had HUGE thermal flctuations; on the order of 15 cm precip in a day
    if Glacier + Station=="Wolverine990":
        end_stage_extra_noise_date='2007-09-18 00'
        #Consider finding a way to folter these huge daily swings into a useful product; currently, option to NAN and trust historical data processing instead
        dat.loc[:end_stage_extra_noise_date, 'StageCumulative']=pd.np.nan #LVL1.precip_remove_drain_and_fill(dat.StageCumulative[:end_stage_extra_noise_date], obvious_error_precip_cutoff=200, n_window=4)

In [24]:
if precip_present:
    #Remove unrealistically large instantaneous jumps; create continuous timreseries
    for precip_col in precip_columns:
        if dat[precip_col].isnull().all():
            print("no data in "+ precip_col)
            continue    
        print(precip_col)
        if dat[precip_col].isnull().all(): #if no values in column, skip
            continue
        dat.loc[:,precip_col]=LVL1.precip_remove_drain_and_fill(dat[precip_col], obvious_error_precip_cutoff=obvious_error_precip_cutoff, n_window=4)
    dat_step0=dat[precip_columns].copy() #save for plotting

In [25]:
if precip_present:
    #Plot
    if not dat[precip_columns[0]].isnull().all() and show_plots:
        plt.figure()
        LVL1.plot_comparrison(df_old=dat_original, df_new=dat_step0, data_col_name=precip_columns[0], label_old="original", label_new="drain and fill removed")

In [26]:
if precip_present:
    #Plot stage-type precip gage
    if not dat[precip_columns[1]].isnull().all() and show_plots:
        plt.figure()
        ax=LVL1.plot_comparrison(df_old=dat_original, df_new=dat_step0, data_col_name=precip_columns[1], label_old="original", label_new="drain and fill removed")

In [27]:
if precip_present:
    #Strip high-amplitude noise related to wind with hampel filter
    for precip_col in precip_columns:
        if dat[precip_col].isnull().all():
            continue
        dat.loc[:,precip_col]=LVL1.hampel(dat_step0[precip_col], t0=2)
    dat_step1=dat[precip_columns].copy() #save for plotting

In [28]:
if precip_present:
    #Run filter on daily window, to remove sensor malfunctions
    for precip_col in precip_columns:
        if dat[precip_col].isnull().all():
            continue
        dat.loc[:,precip_col]=LVL1.basic_median_outlier_strip(dat_step1[precip_col],k=96, threshold=10) #remove values > 10 mm (1cm)from daily median val
    dat_step2=dat[precip_columns].copy() #save for plotting

In [29]:
#Plot results
if precip_present:
    if show_plots and not dat_original[precip_columns[0]].isnull().all():
        plt.figure()
        LVL1.plot_comparrison(df_old=dat_step0, df_new=dat_step2, data_col_name=precip_columns[0], label_old="original", label_new="after hampel outlier filter")

In [30]:
if precip_present:
    #Plot results
    if show_plots and not dat_original[precip_columns[1]].isnull().all():
        plt.figure()
        LVL1.plot_comparrison(df_old=dat_step0, df_new=dat_step2, data_col_name=precip_columns[1], label_old="original", label_new="after hampel outlier filter")

**Smooth Data with method from Nayak (2010): **

In [31]:
if precip_present:
    #Smooth Incremental Precip Data, if there are any decreases in the record
    dx=dat[precip_col]-dat[precip_col].shift(1)
    if dx.min()<0:
        for precip_col in precip_columns:
            #Smooth this data with method from Nayak 2010
            if dat[precip_col].isnull().all(): #if no values in column, skip
                continue
            print ("smoothing "+ precip_col)
            precip_interpolated=dat_step2[precip_col].copy()
            dat[precip_col]=LVL1.smooth_precip_Nayak2010(precip_interpolated)
        dat_step3=dat[precip_columns].copy() #save for plotting
    else:
        print("monotonically increasing series")

In [32]:
if precip_present:
    #Plot
    if show_plots and not dat_original[precip_columns[0]].isnull().all():
        plt.figure()
        LVL1.plot_comparrison(df_old=dat_step2, df_new=dat_step3, data_col_name=precip_columns[0], label_old="original", label_new="after smoothing")

In [33]:
if precip_present:
    if show_plots and not dat_original[precip_columns[1]].isnull().all():
        plt.figure()
        LVL1.plot_comparrison(df_old=dat_step2.sample(frac=frac), df_new=dat_step3.sample(frac=frac), data_col_name=precip_columns[1], label_old="original", label_new="after smoothing")

In [34]:
if precip_present:
    #Final Comparrison Plot
    if show_plots and not dat_original[precip_columns[1]].isnull().all():
        plt.figure()
        frac=0.1
        ax=dat_original[precip_columns[1]].sample(frac=frac).plot(color='black', label='original')
        dat_step0[precip_columns[1]].sample(frac=frac).plot(color='blue', ax=ax, label='fills removed')
        dat_step3[precip_columns[1]].sample(frac=frac).plot(color='red', ax=ax, label='final')
        plt.legend()

Final Steps

In [35]:
if precip_present:
    #Set no-data values before TPG installed, and after rocket removed to NAN (not 0)
    for precip_col in precip_columns:
        if dat[precip_col].isnull().all(): #if no values in column, skip
            continue
        if 'TPG' in precip_col.upper():
            dat.loc[:TPG_born, precip_col]=np.nan
        if 'stage' in precip_col.lower():
            dat.loc[Stage_dies:, precip_col]=np.nan

In [36]:
wind_col

''

In [37]:
dat.head(1)

,Tpassive1
local_time,
1997-12-31 15:00:00,NaN


## Save Edited Data

In [38]:
#Create time as text column
local_timezone=pytz.timezone(timezone) #create local timezone object (e.g. AK time)
dat['Local_time']=dat.index.strftime(date_format)#Create column for true local time (as string, not UTC - X hrs)

#Column Names

#Subset for Wolv 370
if (Glacier+Station=='Wolverine370') | (Glacier=='JuneauIcefield'):
        out_columns=dat.columns
save_dat=dat[out_columns] #removes unwanted columns

#Save
save_dat.to_csv(save_pth, index=False, float_format='%g')
#save_dat.to_csv("Q:/Project Data/GlacierData/Benchmark_Program/Gulkana/AllYears/Wx/LVL1/emily/gulkana1725_15minall.csv", index=False, float_format='%g')


In [39]:
print("saved at: " + save_pth)

saved at: Q:/Project Data/GlacierData/Benchmark_Program/Data/JuneauIcefield/AllYears/Wx/LVL1/juneauicefield2121_15minall.csv


In [40]:
dat[precip_columns].plot()

TypeError: Empty 'DataFrame': no numeric data to plot

In [ ]:
#Plot of places where stage is missing
dat['stg_interp']=dat.StageCumulative.interpolate(limit_direction='both')
to_plot=dat.stg_interp[dat.StageCumulative.isnull()]-100

fig=plt.figure()
ax = fig.add_subplot(111)
ln0=plt.plot(dat[precip_columns])

ln4=ax.plot(to_plot , 'r^')
lns = ln0 +ln4
labs = [l.get_label() for l in lns]
labs[0]='raw 15 min data for TAspirated1'
labs[-1]="daily temperature difference > 2.5 degrees C"